In [53]:
from datetime import date
import pandas as pd
from datahub import backtest as bt 

In [49]:
with pd.HDFStore("fund_info.h5") as store:
    symbol_info = pd.read_hdf(store, "symbol_info")
    fund_quotes = pd.read_hdf(store, "fund_quotes")

In [60]:
filtered_fund = symbol_info[
    (symbol_info["purchase_low_limit"] <= 500000) & symbol_info["trade_status"].str.startswith(u"开放申购")]

In [68]:
filtered_fund.head()

,symbol,start_date,expired_date,sub_fee,purchase_fee,redemption_fee,management_fee,sale_fee,trade_status,purchase_start,purchase_low_limit,purchase_confirm,redem_confirm,redem_date
9,000198.OF,2013-05-29 00:00:00.005,1899-12-30 00:00:00.005,None,None,None,0.30,0.08,开放申购|开放赎回,2013-06-03 00:00:00.005,1.0,T+1,T+1,T+2
10,000203.OF,2013-07-24 00:00:00.005,1899-12-30 00:00:00.005,None,None,None,0.33,0.10,开放申购|开放赎回,2013-08-26 00:00:00.005,1.0,T+1,T+1,T+2
12,000210.OF,2013-09-05 00:00:00.005,1899-12-30 00:00:00.005,None,None,None,0.33,0.10,开放申购|开放赎回,2013-09-23 00:00:00.005,10.0,T+1,T+1,T+2
16,000322.OF,2013-12-18 00:00:00.005,1899-12-30 00:00:00.005,None,None,None,0.27,0.08,开放申购|开放赎回,2014-01-02 00:00:00.005,1000.0,T+1,T+1,T+7
18,000324.OF,2013-10-29 00:00:00.005,1899-12-30 00:00:00.005,None,None,None,0.33,0.10,开放申购|开放赎回,2013-11-15 00:00:00.005,10.0,T+1,T+1,T+3


In [62]:
missing_symbols=[]
performance_result = {}
for symbol in filtered_fund["symbol"]:
    if len(quote) != 139:
        continue
    if symbol not in fund_quotes.items:
        missing_symbols.append(symbol)
        continue
    quote = fund_quotes[symbol]
    quote["daily_ret"] = quote["7day_annual_ret"] / 365
    nav = bt.normalize_change(quote["daily_ret"])
    performance = bt.cal_portfolio_performance(nav, date(2017, 1, 3), date(2017, 7, 28), 1.35)
    performance_result[symbol] = performance
performance_result = pd.DataFrame.from_dict(performance_result, orient="index")

In [69]:
fund_summary = performance_result.merge(filtered_fund[["symbol", "management_fee", "sale_fee", "purchase_confirm",
                                        "redem_confirm", "redem_date"]], left_index=True, right_on="symbol")

In [74]:
fund_summary[(fund_summary["sharpe"] >= 10) & (fund_summary["cum_return"] >= 3) & (fund_summary["redem_date"] != "T+7")]

,daily_std,sharpe,cum_return,daily_mean,symbol,management_fee,sale_fee,purchase_confirm,redem_confirm,redem_date
9,0.000652,19.074543,3.293935,0.010616,000198.OF,0.30,0.08,T+1,T+1,T+2
76,0.000842,47.607545,4.898906,0.012944,000621.OF,0.14,0.05,T+1,T+1,T+1
99,0.000921,11.777832,3.215573,0.010481,000681.OF,0.00,0.10,T+1,T+1,T+2
112,0.001155,10.104369,3.255430,0.010550,000709.OF,0.27,0.05,T+1,T+1,T+2
231,0.000514,33.988885,3.549198,0.011039,001529.OF,0.25,0.05,T+1,T+1,T+1
233,0.000702,15.047094,3.201707,0.010457,001624.OF,0.27,0.05,T+1,T+1,T+1
268,0.001526,10.752343,3.493239,0.010949,001987.OF,0.25,0.10,T+1,T+1,T+3
311,0.000738,23.913147,3.558910,0.011054,002759.OF,0.30,0.08,T+1,T+1,T+1
325,0.001063,28.274995,4.257006,0.012099,002912.OF,0.33,0.05,T+1,T+1,T+1
393,0.000878,15.377327,3.346688,0.010705,003585.OF,0.33,0.05,T+1,T+1,T+2


In [79]:
fund_quotes["000621.OF"]

,7day_annual_ret,daily_yield,yield_mean,yield_sd,annual_ret_mean,annual_ret_sd,daily_ret
2017-01-03,4.858,0.5659,1.566649,0.993511,4.733751,0.096625,0.000000
2017-01-04,4.741,0.9481,1.566649,0.993511,4.733751,0.096625,0.012989
2017-01-05,4.736,1.3698,1.566649,0.993511,4.733751,0.096625,0.012975
2017-01-06,4.500,1.2431,1.566649,0.993511,4.733751,0.096625,0.012329
2017-01-09,4.352,1.3099,1.566649,0.993511,4.733751,0.096625,0.011923
2017-01-10,4.511,0.8576,1.566649,0.993511,4.733751,0.096625,0.012359
2017-01-11,4.499,0.9260,1.566649,0.993511,4.733751,0.096625,0.012326
2017-01-12,4.545,1.4540,1.566649,0.993511,4.733751,0.096625,0.012452
2017-01-13,4.606,1.3557,1.566649,0.993511,4.733751,0.096625,0.012619
2017-01-16,4.593,1.3268,1.566649,0.993511,4.733751,0.096625,0.012584
